In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg
import numpy as np
import galois   

In [2]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [4]:
# R1CS

x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [47]:
# R1CS => QAP
target_coeffs = GF(np.array([1,2,3,4,5]))
target_polynomial = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)
# print(U_vector_of_poly_x)

def aggregate_vector_of_polynomials(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

# This is inner product of Matrix and Witness
Ua = aggregate_vector_of_polynomials(U_vector_of_poly_x, witness)
Va = aggregate_vector_of_polynomials(V_vector_of_poly_x, witness)
Wa = aggregate_vector_of_polynomials(W_vector_of_poly_x, witness)

# We can now create h(x)t(x)
hx, remainder = divmod(((Ua*Va) - Wa), target_polynomial)
assert remainder == 0, "remainder is not 0"
Ua * Va == Wa + hx * target_polynomial



True

In [32]:
# Trusted Setup
tau = GF(9)
powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(target_polynomial.degree)]
powers_of_tau_G2 = [multiply(G2, int(tau) ** i) for i in range(target_polynomial.degree)]
t_of_tau = target_polynomial(tau)
target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i * int(t_of_tau)) for i in range(target_polynomial.degree)]

# Prover
print(type(U_vector_of_poly_x[0]))
print(type(witness))


<class 'galois.Poly'>
<class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
